# CMEMS Visualization

## Import packages

For this visualization of a sample <i>index_latest.txt</i> dataset of the Copernicus Marine Environment Monitoring Service, we use the two packages:
* <a href="https://github.com/python-visualization/folium">folium</a> for the visualization and
* <a href="http://www.numpy.org/">numpy</a> for the data reading / processing.

In [68]:
import numpy as np
import folium

# Selection criteria

## Provider

The list dataprovider contains the name of the providers we want to keep for the plot.

In [69]:
dataproviderlist = ['IEO', 'INSTITUTO ESPANOL DE OCEANOGRAFIA', 'SOCIB']

Here we could also add something for the time or space domain.<br>
Easy but not time to do it now.

## Load and prepare data

Since the <i>index_latest.txt</i> is a formatted file, we use the numpy function <a href="http://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html">genfromtxt</a> to extract the data from the document.

In [70]:
indexfile = "./index_latest.txt"
dataindex = np.genfromtxt(indexfile, skip_header=6, unpack=True, delimiter=',', dtype=None, \
              names=['catalog_id', 'file_name', 'geospatial_lat_min', 'geospatial_lat_max',
                     'geospatial_lon_min', 'geospatial_lon_max',
                     'time_coverage_start', 'time_coverage_end', 
                     'provider', 'date_update', 'data_mode', 'parameters'])

To define the position shown on the map, we use the mean of the stored <i>geospatial_lat/lon_min/max</i> for each dataset.

In [71]:
lon_min = dataindex['geospatial_lon_min']
lon_max = dataindex['geospatial_lon_max']
lat_min = dataindex['geospatial_lat_min']
lat_max = dataindex['geospatial_lat_max']
lonmean, latmean = 0.5*(lon_min + lon_max), 0.5*(lat_min + lat_max)

# Select by data provider

We create a list of indices corresponding to the entries with a provider belonging to the list specified at the beginning.

In [72]:
indexlist = []
for np, provider in enumerate(dataindex['provider']):
    matching = [s for s in dataproviderlist if s in provider]
    if matching:
        indexlist.append(np)

Could do intersection of the list, but for that we need to specify the provider name as specified in the index file.

## netCDF file name conventions

The data specifications are coded within the netCDF file name following the conventions:

<p><b>File naming convention in the latest directory:</b></p>
<ul>
    <li>RR_LATEST_XX_YY_CODE_YYYYMMDD.nc</li>
    <li>RR: region bigram</li>
    <li>LATEST: fixed name</li>
    <li>XX: TS (timeserie) or PR (profile)</li>
    <li>YY: data type</li>
    <li>CODE: platform code</li>
    <li>YYYYMMDD: year month day of observations</li>
    <li>.nc: NetCDF file name suffix</li>
    Example: GL_LATEST_PR_GL_58970_20151112.nc
</ul>

<p><b>Data types</b></p>
<ul>
    <li>BA: data from Bathy messages on GTS</li>
    <li>CT: CTD profiles</li>
    <li>DB: Drifting buoys</li>
    <li>FB: FerryBox</li>
    <li>GL: Gliders</li>
    <li>MO: Fixed buoys or mooring time series</li>
    <li>PF: Profiling floats vertical profiles</li>
    <li>RE: Recopesca</li>
    <li>RF: River flows</li>
    <li>TE: data from TESAC messages on GTS</li>
    <li>TS: Thermosalinographs</li>
    <li>XB: XBT or XCTD profiles</li>
</ul>

<p><b>Region bigram</b></p>
<ul>
    <li>GL: Global</li>
    <li>AR: Arctic</li>
    <li>BO: Baltic</li>
    <li>NO: North West Shelf</li>
    <li>IR: IBI (Iberia-Biscay-Ireland)</li>
    <li>MO: Mediterranean</li>
    <li>BS: Black Sea</li>
</ul>

We convert these information to a python dictionary:

In [73]:
regions_lut = dict()
regions_lut['GL'] = 'Global'
regions_lut['AR'] = 'Arctic'
regions_lut['BO'] = 'Baltic'
regions_lut['NO'] = 'North West Shelf'
regions_lut['IR'] = 'IBI (Iberia-Biscay-Ireland)'
regions_lut['MO'] = 'Mediterranean'
regions_lut['BS'] = 'Black Sea'
data_types_lut = dict()
data_types_lut['BA'] = 'data from Bathy messages on GTS'
data_types_lut['CT'] = 'CTD profiles'
data_types_lut['DB'] = 'Drifting buoys'
data_types_lut['FB'] = 'FerryBox'
data_types_lut['GL'] = 'Gliders'
data_types_lut['MO'] = 'Fixed buoys or mooring time series'
data_types_lut['PF'] = 'Profiling floats vertical profiles'
data_types_lut['RE'] = 'Recopesca'
data_types_lut['RF'] = 'River flows'
data_types_lut['TE'] = 'data from TESAC messages on GTS'
data_types_lut['TS'] = 'Thermosalinographs'
data_types_lut['XB'] = 'XBT or XCTD profiles'
data_specs_lut = dict()
data_specs_lut['TS'] = 'Timeseries'
data_specs_lut['PR'] = 'Profile'

## Visualization

Finally, we create the map object.

In [74]:
map = folium.Map(location=[39.5, 2], zoom_start=8)
cntr = 0
for i in indexlist:
    curr_data = dataindex[i]
    link = curr_data[1]
    
    last_idx_slash = link.rfind('/')
    
    ncdf_file_name = link[last_idx_slash+1::]
    
    if ncdf_file_name[10:12] in data_specs_lut:
        data_spec = data_specs_lut[ncdf_file_name[10:12]]
    else:
        data_spec = ncdf_file_name[10:12]
    if ncdf_file_name[13:15] in data_types_lut:    
        data_type = data_types_lut[ncdf_file_name[13:15]]
    else:
        data_type = ncdf_file_name[13:15]
    if ncdf_file_name[0:2] in regions_lut:
        region = regions_lut[ncdf_file_name[0:2]]
    else:
        region = ncdf_file_name[0:2]
    
    platform_code = ncdf_file_name[16:-12]
    #observation_date = ncdf_file_name[-11:-3]
    provider = curr_data['provider']
    data_parameters = curr_data['parameters']
    
    time_start = curr_data['time_coverage_start']
    time_end = curr_data['time_coverage_end']
    
    popup_html = """
    <table border=0 width=300px>
        <tr>
            <td width="40%">Platform Code</td>
            <td width="60%">{platform_code}</td>
        </tr>
        <tr>
            <td>Provider</td>
            <td>{provider}</td>
        </tr>
        <tr>
            <td>Type of Data</td>
            <td>{data_spec}</td>
        </tr>
        <tr>
            <td>Region</td>
            <td>{region}</td>
        </tr>
        <tr>
            <td>Data Information</td>
            <td>{data_type}</td>
        </tr>
        <tr>
            <td>Provided Data</td>
            <td>{data_parameters}</td>
        </tr>
        <tr>
            <td>Time Coverage Start</td>
            <td>{time_start}</td>
        </tr>
        <tr>
            <td>Time Coverage End</td>
            <td>{time_end}</td>
        </tr>
        <tr>
            <td>NetCDF File</td>
            <td><a href="{link}">FTP Server Link</a></td>
        </tr>        
    </table>
    """.format(platform_code=platform_code, provider=provider, data_spec=data_spec, region=region,
               data_type=data_type, data_parameters=data_parameters, time_start=time_start,
               time_end=time_end, link=link)
    map.simple_marker( location = [latmean[i], lonmean[i]], clustered_marker = True, popup=popup_html)

Add some tiles to the dataset.

In [75]:
map.add_tile_layer(tile_name='World Ocean Base', tile_url='http://services.arcgisonline.com/arcgis/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}')
map.add_tile_layer(tile_name='World Topo Map', tile_url='http://services.arcgisonline.com/arcgis/rest/services/World_Topo_Map/MapServer/MapServer/tile/{z}/{y}/{x}')
map.add_tile_layer(tile_name='World Ocean Reference', tile_url='http://services.arcgisonline.com/arcgis/rest/services/Ocean/World_Ocean_Reference/MapServer/tile/{z}/{y}/{x}')
map.add_layers_to_map()

In [76]:
map

In [ ]:
map.create_map(path='CMEMS_latest_index.html')